# Knowledge base System
문서를 이용한 벡터DB 구성 실습

In [1]:
# 원본 tax.docx

# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_pure.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma_origin' # 저장소 이름 다르게 해
)

# Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

AIMessage(content='연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 다음의 절차를 따릅니다.\n\n1. **과세표준 계산**: 직장인의 소득은 근로소득에 해당합니다. 근로소득금액은 총급여액에서 비과세소득을 제외하고, 근로소득공제를 적용하여 계산합니다. \n\n2. **근로소득공제**: 연봉에 따라 근로소득공제는 다음과 같이 적용됩니다. (2024년 기준)\n   - 연봉 5천만원에 대한 근로소득공제는 대략 1,500만원(상한선 기준)일 수 있습니다. \n\n3. **과세표준**: \n   \\[\n   5,000만원 - 1,500만원 = 3,500만원\n   \\]\n\n4. **세율 적용**: 과세표준에 따라 소득세를 계산하는 단계입니다.\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 4,600만원 이하: 15%\n   - 4,600만원 초과 8,800만원 이하: 24%\n\n5. **세액 계산**: \n   \\[\n   0~1,200만원: 1,200만원 × 6\\% = 72만원\n   \\]\n   \\[\n   1,200만원~3,500만원 (2,300만원 적용): 2,300만원 × 15\\% = 345만원\n   \\]\n   \\[\n   \\text{총소득세} = 72만원 + 345만원 = 417만원\n   \\]\n\n따라서, 연봉 5천만원인 직장인의 소득세는 약 417만원으로 예상됩니다. 다만, 최종적으로는 개인의 비과세소득이나 공제항목에 따라 달라질 수 있습니다. 추가적인 공제나 세액감면이 있을 경우 세액은 줄어들 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 441, 'prompt_tokens': 5150, 'total_tokens': 5591, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio

In [1]:
# 표 마크타운으로 수정
# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma_new' # 저장소 이름 다르게 해
)

# Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

AIMessage(content='연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 해당 금액이 어디에 해당하는지 종합소득 과세표준 구간을 확인해야 합니다.\n\n연봉 5천만 원은 다음과 같은 구간에 해당합니다:\n- 1,400만 원 초과 ~ 5,000만 원 이하\n\n이 구간의 세율에 따른 소득세 계산식은 다음과 같습니다:\n- 84만 원 + (1,400만 원 초과분 × 15%)\n\n이제 계산을 해보겠습니다.\n\n1. 5천만 원 - 1,400만 원 = 3,600만 원 (1,400만 원 초과분)\n2. 3,600만 원 × 15% = 540만 원\n3. 총 소득세 = 84만 원 + 540만 원 = 624만 원\n\n따라서 연봉 5천만 원인 직장인의 소득세는 **624만 원**입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 219, 'prompt_tokens': 5081, 'total_tokens': 5300, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDoIyzL0X6tpjHsT9IL2i7OcBgrvu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--02784333-d32e-47d6-8e83-801ec35e132a-0', usage_metad